# 将教材处理成JSON格式文件的pipline脚本
## 基本思路
首先将PDF教材人工分页，每个部分内容围绕同一个topic，然后用百度文库变成word版本。之后调用大模型进行chunking+processing

## chunking

### chunking procedure

In [13]:
import dashscope
import re
import json
def is_new_disease_start(paragraph):
    title_pattern = r'^[一二三四五六七八九十 ]+、'
    if re.match(title_pattern, paragraph):
        return True
    else:
        return False
# 假设 call_large_model 是一个函数，用于调用大模型并返回生成的字符串
def call_large_model(prompt):
    messages=[{"role": "system", "content": "你是一个医学知识问答机器人。"},{"role": "user", "content": prompt}]
    response = dashscope.Generation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-969f4200d53442a2a1733d1c0b1fb330",
    model="qwen-plus", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=messages,
    result_format='message'
    )
    return response['output']['choices'][0]['message']['content'].strip()
def dynamic_chunking(paragraphs):
    """
    使用大模型动态判断段落是否是新疾病的开始，从而进行分块。
    
    :param paragraphs: 文档的所有段落
    :return: 疾病文本列表
    """
    chunked_diseases = []  # 存储最终的疾病文本
    current_disease = ""  # 当前疾病的累积文本
    flag=False
    for paragraph in paragraphs:
        # 判断当前段落是否是一个新疾病的开始
        if is_new_disease_start(paragraph):
            print(paragraph)
            flag=True
            if current_disease:  # 如果已有疾病文本，保存到列表
                chunked_diseases.append(current_disease.strip())
            current_disease = paragraph  # 开始新的疾病描述
        else:
            if flag:
                current_disease += "\n" + paragraph  # 继续追加到当前疾病描述
    # 添加最后一个疾病
    if current_disease:
        chunked_diseases.append(current_disease.strip())
    return chunked_diseases
def generate_json_block(disease_text):
    prompt = f"""
    请将以下疾病的描述转化为JSON格式的block。
    要求：
    1.输出的JSON的内容应该完全忠实于给定文本，将给定文本按照结构层次提取相应的属性，如诊断要点、鉴别诊断、疾病定义、疾病特性描述、治疗要点、预防、预后等。
    2.只输出JSON block中的内容，不要输出任何额外的信息。包括json block的开始和结束标记。```json```
    3.只输出一个JSON block。鉴别诊断里可能包含其他疾病的描述，请在鉴别诊断的属性中输出这些疾病的JSON block。
    疾病描述：
    {disease_text}
    """
    
    # 调用大模型 API
    json_block = call_large_model(prompt)
    print(json_block)
    # 将生成的JSON字符串解析为Python字典
    return json.loads(json_block)

In [14]:
from docx import Document

# 假设Word文档路径为 '1.doc'
doc = Document('/home/lym/GraphRAG4OralHealth/OralGraph/ClassifyTree/疾病诊断及治疗原则11738846462.docx')
# 初始化结果列表
json_blocks = []

# Step 1: 提取段落
paragraphs = [para.text.strip() for para in doc.paragraphs if para.text.strip()]

# Step 2: 使用动态分块机制进行Chunking
chunked_diseases = dynamic_chunking(paragraphs)

# Step 3: 对每个疾病文本生成JSON block
for disease_text in chunked_diseases:
    json_block = generate_json_block(disease_text)
    json_blocks.append(json_block)

# 输出所有JSON block
for block in json_blocks:
    print(json.dumps(block, ensure_ascii=False, indent=4))

一 、复发性阿弗他溃疡
三、白塞综合征
四、放射性口腔黏膜炎
五、化疗性口腔黏膜炎
二 、单纯疱疹
三、带状疱疹
四 、手足口病
五、口腔念珠菌病
六、疱疹性咽峡炎
七、口腔结核
一 、口腔白斑病
三 、口腔扁平苔藓
四、口腔白色角化症
五 、白色海绵状斑痣
六、盘状红斑狼疮
七、口腔黏膜下纤维化
一、天疱疮
二 、黏膜类天疱疮
三、 副肿瘤性天疱疮
四、 扁平苔藓样类天疱疮
一 、慢性唇炎
二 、光化性唇炎
三 、腺性唇炎
四、肉芽肿性唇炎
五 、梅 -罗综合征
六、口角炎
二、沟 纹 舌
五、正中菱形舌
六、灼口综合征
一 、药物过敏性口炎
二、血管性水肿
三 、多 形 红 斑
四 、接触性过敏性口炎
一 、贫 血
二 、血小板减少性紫癜
三、白血病
四、核黄素缺乏症
五、色素沉着息肉综合征
六、干燥综合征
七、结节病
八、淀粉样变性
九 、糖尿病
一 、生理性色素沉着
二 、口腔黏膜黑斑
三 、口腔黑棘皮病
四、原发性慢性肾上腺皮质功能减退症
五、银汞文身
六、重金属色素沉着
七、烟草及药物引起的色素沉着
一 、梅 毒
二、淋病
三、尖锐湿疣
四 、艾 滋 病
```json
{
  "疾病名称": "复发性阿弗他溃疡",
  "疾病定义": "复发性阿弗他溃疡(recurrent aphthous ulcer, RAU),又称为复发性口腔溃疡(recurrent oral ulcer, ROU)、复发性阿弗他口炎(recurrent aphthous stomatitis, RAS)，是最常见的口腔黏膜溃疡类疾病。",
  "疾病特性描述": "在一般人群中发病率为20%左右，特定人群可高达50%。病因尚不明确，治疗无根治方法。具有典型的周期性、反复发作、可自愈的特征。",
  "诊断要点": "根据病史及临床表现即可做出诊断，但需除外系统性疾病引起的口腔溃疡。",
  "临床表现": {
    "轻型阿弗他溃疡": {
      "临床表现": "1)口腔黏膜尤其是非角化黏膜可见圆形、椭圆形的孤立或散在溃疡，直径<10mm。2)数目一般1~5个不等。3)溃疡呈典型“红、黄、凹、痛”特点，即浅碟状，表面有黄白色假膜，周围有窄的红晕，溃疡局部疼痛。4)溃疡愈合期一般为7~14天，愈合后不留瘢痕。5)间歇期因人而异，长短不一，呈周期性反复发作。",
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)